In [ ]:
import math
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

from spinorama.graph_isobands import find_isobands


def transform_id_x(x, y):
    return x


def transform_id_y(x, y):
    return y

In [ ]:
p0 = (0, 0)
p1 = (2, 0)
p2 = (2, 1)
p3 = (0, 1)

x = [[0, 2], [0, 2]]
y = [[0, 1], [2, 1]]
z = [[3, 2], [2, 3.2]]

df = find_isobands(x, y, z, [0, 1.5, 2.5, 3], transform_id_x, transform_id_y)

g_df = (
    alt.Chart(alt.Data(values=df["features"]))
    .mark_geoshape(stroke="red", strokeWidth=2, clip=True, cornerRadius=2)
    .encode(
        alt.Color(
            "properties.z_low:Q", scale=alt.Scale(scheme="bluegreen"), title="IsoBands"
        )
    )
    .project("identity")
    .properties(width=200, height=200)
)

g_df

In [ ]:
xr = np.linspace(-5, 5, 30)
yr = np.linspace(-5, 5, 30)
x, y = np.meshgrid(xr, yr)
z = x ** 2 + y ** 2
print(len(xr), len(yr), z.shape)

source = pd.DataFrame({"x": x.ravel(), "y": y.ravel(), "z": z.ravel()})

df = find_isobands(x, y, z, [1, 2, 4, 8, 16, 32], transform_id_x, transform_id_y)

g_source = alt.Chart(source).mark_rect().encode(x="x:O", y="y:O", color="z:Q")

g_iso = (
    alt.Chart(alt.Data(values=df["features"]))
    .mark_geoshape(stroke="red", strokeWidth=0, clip=True, cornerRadius=0)
    .encode(
        alt.Color(
            "properties.z_low:Q", scale=alt.Scale(scheme="bluegreen"), title="IsoBands"
        )
    )
    .project("identity")
)

# g_source+g_iso
g_iso

In [ ]:
from spinorama.load import graph_melt
import flammkuchen as fl
from spinorama.graph_contour import compute_contour
from spinorama.graph import contour_params_default, graph_contour_common

df_all = fl.load("../cache.parse_all_speakers.h5")

In [ ]:
df = df_all["Genelec 8341A"]["ASR"]["asr"]
print(df.keys())

In [ ]:
dfu = df["SPL Vertical_unmelted"]
af, am, az = compute_contour(dfu.loc[dfu.Freq > 100])
print(len(af[0]), len(am[:, 0]), az.shape)
print(af[0][0:3])
print(am[:, 0])

In [ ]:
graph_width = 1024
graph_height = 720


def transform_log(x, y):
    return 6 * np.log10(x) * graph_width / 800


def transform_radian(x, y):
    return 1.325 * y / 180 * math.pi * graph_height / 360


bands = [-72, -18, -15, -12, -9, -6, -3, +3]
df_iso = find_isobands(af, am, az.T, bands, transform_log, transform_radian)

color_legend = [
    "[{0}, {1}]".format(bands[i], bands[i + 1]) for i in range(0, len(bands) - 1)
] + [">{0}".format(bands[-1])]
color_range = [
    "black",
    "blue",
    "steelblue",
    "green",
    "yellow",
    "orange",
    "red",
    "white",
]

print(color_legend)

isobands = (
    alt.Chart(alt.Data(values=df_iso["features"]))
    .mark_geoshape(stroke="red", strokeWidth=1)
    .encode(
        # alt.Color('properties.z_low:Q', scale=alt.Scale(scheme='inferno'), title = "IsoBands")
        alt.Color(
            "properties.z_low:O",
            scale=alt.Scale(zero=False, domain=bands, range=color_range),
            legend=alt.Legend(orient="bottom", title="Relative dB SPL"),
        )
    )
    .project("identity")
    .properties(width=graph_width, height=graph_height)
)

In [ ]:
empty = pd.DataFrame({"Freq", "Angle"})
yTicks = [-180 + 30 * i for i in range(0, 13)]
print(yTicks)
legends = (
    alt.Chart(empty)
    .mark_point(clip=True)
    .encode(
        x=alt.X(
            "x:Q",
            scale=alt.Scale(type="log", nice=False, domain=[20, 20000]),
            title="Frequency (Hz)",
        ),
        y=alt.Y(
            "y:Q",
            scale=alt.Scale(nice=False, domain=[-180, 180]),
            axis=alt.Axis(format=".0d", values=yTicks, title="Angle"),
        ),
    )
    .properties(width=graph_width, height=graph_height)
)

In [ ]:
isobands + legends

In [ ]:
def transform_polar_r(x, y):
    a = y * math.pi / 180
    return np.log10(x) * np.cos(a)


def transform_polar_theta(x, y):
    a = y * math.pi / 180
    return np.log10(x) * np.sin(a)


df_polar = find_isobands(
    af, am, az.T, bands, transform_polar_r, transform_polar_theta, False, True
)
alt.Chart(alt.Data(values=df_polar["features"])).mark_geoshape(strokeWidth=0).encode(
    alt.Color(
        "properties.z_low:O",
        scale=alt.Scale(zero=False, domain=bands, range=color_range),
        legend=alt.Legend(orient="bottom", title="Relative dB SPL"),
    )
).project("identity").properties(width=graph_width, height=graph_height)